#### Bayesian Statistics Interactive

Bayes' theorem is shown here using an example. Individual probabilities can be changed interactively so that the changes to the end result are immediately visible.

Bayes' theorem:
For two events A and B, for B ≠ 0, Bayes' theorem is:

$$ {P(A|B)} = \frac{P(B|A) \bullet P(A)}{P(B)}$$

P(A | B) is the (conditional) probability of event A under the condition that B has occurred. P(B | A) is the (conditional) probability of event B under the condition that A has occurred
P(A) is the probability (initial probability) that event A will occur
P(B) is the probability (initial probability) that event B will occur
Initial probability means that one event is viewed independently of another.

In [6]:
#resourcen

import ipywidgets as widgets
import numpy as np


In [7]:
# interactive switches

# layout limitations
layout_right_text = {'width': '20%'}
layout_left_text = {'width': '40%'}

# Reliability of detection if patient falls ill
pB_A = widgets.BoundedFloatText(value=99, min=0.01, max=100.0, step=0.01, layout=layout_right_text)

# Percentage of positive tests in healthy patients
pB_Anot = widgets.BoundedFloatText(value=0.05, min=0.01, max=100.0, step=0.01, layout=layout_right_text)

#
pA = widgets.BoundedFloatText(value=0.01, min=0.01, max=100.0, step=0.01, layout=layout_right_text)

#
p_spreading = widgets.HBox(
    [
        widgets.Label(value="spread of the disease (%):", layout=layout_left_text),
        pA
    ]
)
#
p_verify = widgets.HBox(
    [
        widgets.Label(
            value='Verifiability of the test when the patient is ill (%):', layout=layout_left_text),
        pB_A        
    ]
)
#
p_error = widgets.HBox(
    [
        widgets.Label(value='Error rate of the test if the patient is not ill  (%):', layout=layout_left_text),
        pB_Anot
    ]
)
#
style = {'description_width': 'initial'}
#
outpriori = widgets.VBox(
[
    # P (ill) = pA
    widgets.Box([widgets.Label(value="P(ill) (%)", layout=layout_left_text),
                widgets.FloatText(description = "", value = pA.value ,disabled = True,\
                                  layout=layout_right_text, style=style)]
                ),
    # P (not ill) = 1-pA
    widgets.Box([widgets.Label(value="P(not ill) (%)", layout=layout_left_text),
                 widgets.FloatText(value = (100 - pA.value), layout=layout_right_text,\
                                   disabled = True, style=style)]
               ),
    # P (test _positive | ill) = pB_ A (p detection reliability)
    widgets.Box([widgets.Label(value= "P(test positive | ill) (%)", layout=layout_left_text),
                 widgets.FloatText(value = '%2.4f' % (pB_A.value),\
                                  layout=layout_right_text,disabled = True, style=style)]
               ),
    # P (test_positive | not ill) = pB_Anot
    widgets.Box([widgets.Label(value="P(test positive | not ill) (%)", layout=layout_left_text),
                 widgets.FloatText(value = '%2.4f' % (pB_Anot.value),\
                                   layout=layout_right_text, disabled = True, style=style)]
               )
])
# P(ill|test_positive) = [pA * pB_A] / [pA * pB_A + pA_not * pB_Anot]
outposteriori = widgets.VBox(
[
    widgets.Label(value="Probability that a patient is ill if the test is positive  (%):"),
    widgets.FloatText(value = '%2.4f' % ((pB_A.value * pA.value) / \
                                                (pB_A.value * pA.value + pB_Anot.value * (100 - pA.value))*100),\
                      description="", disabled = True,layout={'width': '30%'})
])
#
BtnReset = widgets.Button(description="reset", button_style='warning')
#

In [8]:
# update function that is executed when a switch is changed
#
def update_view(*args):
    """ Probability that a patient is ill,\
        if the test is positive, recalculate with the given parameters.
        
        P (diseased) = pA
        P (not diseased) = 1-pA
        P (test_ positive | sick) = PA *pB_A (p detection security)
        P (test_positiv |not diseased) = P (not diseased)* perror rate (pB_Anot)
        P (diseased | test_positive) = [pA *pDetection security] / [pA* pDetection security + pA_not *perror rate]
                                                                {=> P (B) calculated over total probability}
        """
    outpriori.children[0].children[1].value = pA.value
    outpriori.children[1].children[1].value = (100 - pA.value)
    outpriori.children[2].children[1].value = '%3.4f' % (pB_A.value)
    outpriori.children[3].children[1].value = '%3.4f' % (pB_Anot.value)
    #
    # Intercept extreme values
    if (pB_Anot.value == 0):
        outposteriori.children[1].value = 0.0
        return
    if (pA.value == 100):
        outposteriori.children[1].value = 100
        return
    outposteriori.children[1].value = '%2.4f' % ((pB_A.value * pA.value) / \
                                                 (pB_A.value * pA.value + pB_Anot.value * (100-pA.value))*100)
    #

In [9]:
# Reset settings to default values
#
def reset (* btn):
    pA.value = 0.01
    pB_A.value = 99
    pB_Anot.value= 0.05
    update_view()

Bayes' theorem is shown here with interactive widgets.<br>Individual probabilities can be changed interactively so that the changes to the end result are immediately visible.


In this example, a virtual medical test is taken to demonstrate Bayes' thoerem with the question:
 **Is a patient ill when the test is positive?**

Interactive inputs:
- If the patient is diseased, the test has a probability of 0.xx. (default: 0.99)
- in x cases the test is positive, even if the patient is not diseased (default: 0.005)
- Prior information, i.e. how widespread the disease is (default: 0.0001)

The conditional probabilities are first calculated from this:
- P(diseased)
- P(not diseased)
- P(test positive | diseased)
- P(test positive | not diseased)

And then by means of the Bayesian theorem P(diseased | test positive) - what we want to know.


In [10]:
# Add update function
#
pA.observe(update_view, 'value')
pB_A.observe(update_view, 'value')
pB_Anot.observe(update_view, 'value')
BtnReset.on_click(reset)
#
# Arrange switch
#
widgets.VBox([widgets.HTML(value="<hr>"),
              widgets.VBox([p_spreading, p_verify, p_error]),
              widgets.HTML(value="<hr>"),
              outpriori,
              widgets.HTML(value="<hr>"),
              outposteriori,
              widgets.HTML(value="<hr>"),
              BtnReset])

Copyright © 2020 IUBH Internationale Hochschule